# 니켈 가격 예측 - 자율방식 성능 향상 실험

---

## 🎯 목표
기존 분석(sparta2.ipynb) 대비 **예측 성능 향상**

## 📋 실험 전략

### 1. 고급 피처 엔지니어링
- 기술적 지표 (RSI, MACD, Bollinger Bands)
- 다중 지연 피처 (lag 1, 2, 4, 8주)
- 롤링 통계량 (평균, 표준편차, 왜도, 첨도)
- 피처 상호작용

### 2. 고급 모델링
- 하이퍼파라미터 최적화 (Optuna)
- 앙상블 (Voting, Stacking with meta-learner)
- 시계열 교차검증 (TimeSeriesSplit)

### 3. 하이브리드 전략
- Naive + ML 동적 가중치
- 시장 레짐 기반 모델 전환

---

## STEP 0. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML 라이브러리
from sklearn.ensemble import (
    GradientBoostingRegressor, RandomForestRegressor,
    VotingRegressor, StackingRegressor
)
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# 설정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print('환경 설정 완료')

## STEP 1. 데이터 로드 및 기본 전처리

In [ ]:
# 기존 데이터 로드
df_raw = pd.read_csv('data_weekly_260120.csv')
df_raw['dt'] = pd.to_datetime(df_raw['dt'])
df_raw = df_raw.set_index('dt').sort_index()

# 타겟 정의
target_col = 'Com_LME_Ni_Cash'

# 결측치 처리
df = df_raw.ffill().bfill()

print(f'데이터 크기: {df.shape}')
print(f'기간: {df.index.min()} ~ {df.index.max()}')
print(f'타겟 변수: {target_col}')

---

## STEP 2. 고급 피처 엔지니어링

### 2.1 기술적 지표 생성
- RSI (Relative Strength Index): 과매수/과매도 지표
- MACD: 추세 및 모멘텀
- Bollinger Bands: 변동성 기반 밴드

In [ ]:
def create_technical_indicators(df, price_col):
    """
    기술적 지표 생성
    """
    result = df.copy()
    price = df[price_col]
    
    # 1. RSI (14주 기준)
    delta = price.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    result[f'{price_col}_RSI'] = 100 - (100 / (1 + rs))
    
    # 2. MACD
    ema12 = price.ewm(span=12).mean()
    ema26 = price.ewm(span=26).mean()
    result[f'{price_col}_MACD'] = ema12 - ema26
    result[f'{price_col}_MACD_signal'] = result[f'{price_col}_MACD'].ewm(span=9).mean()
    result[f'{price_col}_MACD_hist'] = result[f'{price_col}_MACD'] - result[f'{price_col}_MACD_signal']
    
    # 3. Bollinger Bands (20주 기준)
    sma20 = price.rolling(20).mean()
    std20 = price.rolling(20).std()
    result[f'{price_col}_BB_upper'] = sma20 + 2 * std20
    result[f'{price_col}_BB_lower'] = sma20 - 2 * std20
    result[f'{price_col}_BB_width'] = (result[f'{price_col}_BB_upper'] - result[f'{price_col}_BB_lower']) / sma20
    result[f'{price_col}_BB_position'] = (price - result[f'{price_col}_BB_lower']) / (result[f'{price_col}_BB_upper'] - result[f'{price_col}_BB_lower'] + 1e-10)
    
    # 4. 이동평균
    for window in [4, 8, 12, 26]:
        result[f'{price_col}_SMA{window}'] = price.rolling(window).mean()
        result[f'{price_col}_EMA{window}'] = price.ewm(span=window).mean()
    
    # 5. 모멘텀
    for lag in [1, 2, 4, 8]:
        result[f'{price_col}_momentum_{lag}w'] = price / price.shift(lag) - 1
    
    return result

# 니켈에 대해 기술적 지표 생성
df_tech = create_technical_indicators(df, target_col)
print(f'기술적 지표 추가 후 컬럼 수: {df_tech.shape[1]}')

### 2.2 다중 지연 피처 (Multi-lag Features)

In [ ]:
def create_lag_features(df, cols, lags=[1, 2, 4, 8]):
    """
    다중 지연 피처 생성
    """
    result = df.copy()
    for col in cols:
        for lag in lags:
            result[f'{col}_lag{lag}'] = df[col].shift(lag)
    return result

# 주요 변수에 대해 지연 피처 생성
key_cols = [
    'Com_LME_Cu_Cash', 'Com_LME_Zn_Cash', 'Com_LME_Pb_Cash',
    'Com_Iron_Ore', 'Com_Gold',
    'EX_USD_CNY', 'EX_USD_JPY',
    'Idx_SP500', 'Idx_VIX'
]
existing_cols = [c for c in key_cols if c in df_tech.columns]
df_lag = create_lag_features(df_tech, existing_cols, lags=[1, 2, 4])
print(f'지연 피처 추가 후 컬럼 수: {df_lag.shape[1]}')

### 2.3 롤링 통계량

In [ ]:
def create_rolling_features(df, price_col, windows=[4, 8, 12]):
    """
    롤링 통계량 생성
    """
    result = df.copy()
    price = df[price_col]
    
    for w in windows:
        result[f'{price_col}_roll_mean_{w}'] = price.rolling(w).mean()
        result[f'{price_col}_roll_std_{w}'] = price.rolling(w).std()
        result[f'{price_col}_roll_min_{w}'] = price.rolling(w).min()
        result[f'{price_col}_roll_max_{w}'] = price.rolling(w).max()
        result[f'{price_col}_roll_skew_{w}'] = price.rolling(w).skew()
        result[f'{price_col}_roll_range_{w}'] = result[f'{price_col}_roll_max_{w}'] - result[f'{price_col}_roll_min_{w}']
    
    return result

df_roll = create_rolling_features(df_lag, target_col)
print(f'롤링 통계량 추가 후 컬럼 수: {df_roll.shape[1]}')

### 2.4 피처 최종 정리

In [ ]:
# 최종 데이터셋
df_final = df_roll.copy()

# 타겟 분리
y = df_final[target_col]

# 피처에서 타겟 제외 + 1주 지연 (누수 방지)
feature_cols = [c for c in df_final.columns if c != target_col]
X = df_final[feature_cols].shift(1)

# 결측치 제거 (처음 몇 주는 롤링/지연으로 인해 NaN)
valid_idx = X.dropna().index.intersection(y.dropna().index)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

print(f'최종 데이터: {X.shape[0]}개 샘플, {X.shape[1]}개 피처')
print(f'기간: {X.index.min().date()} ~ {X.index.max().date()}')

---

## STEP 3. 데이터 분할

기존과 동일한 기간 사용 (공정한 비교를 위해)

In [ ]:
# 기간 설정 (기존과 동일)
val_start = pd.to_datetime('2025-08-04')
test_start = pd.to_datetime('2025-10-27')

# 분할
train_mask = X.index < val_start
val_mask = (X.index >= val_start) & (X.index < test_start)
test_mask = X.index >= test_start

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]
X_test, y_test = X[test_mask], y[test_mask]

print(f'Train: {len(X_train)} samples ({X_train.index.min().date()} ~ {X_train.index.max().date()})')
print(f'Val: {len(X_val)} samples')
print(f'Test: {len(X_test)} samples')

In [ ]:
# 데이터 분할 시각화
fig, ax = plt.subplots(figsize=(14, 4))

ax.plot(y.index, y.values, 'k-', linewidth=0.8, alpha=0.5)
ax.axvspan(y_train.index.min(), y_train.index.max(), alpha=0.3, color='blue', label='Train')
ax.axvspan(y_val.index.min(), y_val.index.max(), alpha=0.3, color='green', label='Validation')
ax.axvspan(y_test.index.min(), y_test.index.max(), alpha=0.3, color='red', label='Test')

ax.set_title('Data Split', fontweight='bold')
ax.set_ylabel('Nickel Price')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## STEP 4. 피처 선택 (SHAP 기반)

Train 데이터만 사용하여 중요 피처 선택

In [ ]:
import shap

# XGBoost로 SHAP 계산
model_shap = xgb.XGBRegressor(n_estimators=100, random_state=RANDOM_SEED, n_jobs=-1)
model_shap.fit(X_train, y_train)

explainer = shap.TreeExplainer(model_shap)
shap_values = explainer.shap_values(X_train)

# 중요도 계산
importance = np.abs(shap_values).mean(axis=0)
feat_imp = pd.DataFrame({
    'feature': X_train.columns,
    'importance': importance
}).sort_values('importance', ascending=False)

# 상위 30개 피처 선택
top_n = 30
selected_features = feat_imp.head(top_n)['feature'].tolist()

print(f'선택된 피처 수: {len(selected_features)}')
print('\n상위 10개 피처:')
for i, row in feat_imp.head(10).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

In [ ]:
# 피처 중요도 시각화
fig, ax = plt.subplots(figsize=(10, 8))

top_20 = feat_imp.head(20).sort_values('importance')
ax.barh(top_20['feature'], top_20['importance'], color='teal')
ax.set_xlabel('SHAP Importance')
ax.set_title('Top 20 Feature Importance (Advanced Features)', fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
# 선택된 피처로 데이터셋 재구성
X_train_sel = X_train[selected_features]
X_val_sel = X_val[selected_features]
X_test_sel = X_test[selected_features]

print(f'최종 피처 수: {len(selected_features)}')

---

## STEP 5. 모델 학습 및 최적화

### 5.1 평가 함수 정의

In [ ]:
def evaluate(y_true, y_pred):
    """모델 평가 함수"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

def print_eval(name, metrics):
    print(f"{name}: RMSE={metrics['RMSE']:.2f}, MAE={metrics['MAE']:.2f}, MAPE={metrics['MAPE']:.2f}%")

### 5.2 Baseline: Naive 모델

In [ ]:
# Naive 모델 (기존과 동일)
prev_price = y.shift(1).loc[y_test.index]
prev_prev = y.shift(2).loc[y_test.index]

# 첫 시점 처리
last_train = y.loc[y.index < y_test.index[0]]
prev_price.iloc[0] = last_train.iloc[-1]
prev_prev.iloc[0] = last_train.iloc[-2]

naive_last = prev_price
naive_drift = prev_price + (prev_price - prev_prev)

print('=== Baseline (Naive) ===' )
print_eval('Naive_Last', evaluate(y_test, naive_last))
print_eval('Naive_Drift', evaluate(y_test, naive_drift))

baseline_rmse = evaluate(y_test, naive_drift)['RMSE']
print(f'\n→ 목표: RMSE < {baseline_rmse:.2f} (Naive_Drift)')

### 5.3 ML 모델 학습 (개별)

In [ ]:
# 스케일링
scaler = RobustScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_sel),
    index=X_train_sel.index,
    columns=X_train_sel.columns
)
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_sel),
    index=X_val_sel.index,
    columns=X_val_sel.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_sel),
    index=X_test_sel.index,
    columns=X_test_sel.columns
)

# 전체 학습 데이터 (Train + Val)
X_full = pd.concat([X_train_scaled, X_val_scaled])
y_full = pd.concat([y_train, y_val])

print('스케일링 완료')

In [ ]:
# 개별 모델 정의
models = {
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1, verbose=-1
    ),
    'CatBoost': CatBoostRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, verbose=0
    ),
    'Ridge': Ridge(alpha=1.0),
    'RandomForest': RandomForestRegressor(
        n_estimators=200, max_depth=6, random_state=RANDOM_SEED, n_jobs=-1
    )
}

# 학습 및 평가
results = {}
predictions = {}

print('=== 개별 모델 성능 (Test) ===')
for name, model in models.items():
    model.fit(X_full, y_full)
    pred = model.predict(X_test_scaled)
    metrics = evaluate(y_test, pred)
    results[name] = metrics
    predictions[name] = pred
    print_eval(name, metrics)

### 5.4 앙상블 모델

In [ ]:
# Voting Regressor (상위 3개 모델)
top_models = sorted(results.items(), key=lambda x: x[1]['RMSE'])[:3]
print(f'상위 3개 모델: {[m[0] for m in top_models]}')

voting = VotingRegressor([
    ('gb', GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED)),
    ('xgb', xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1)),
    ('lgbm', lgb.LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1, verbose=-1))
])

voting.fit(X_full, y_full)
pred_voting = voting.predict(X_test_scaled)
results['Voting'] = evaluate(y_test, pred_voting)
predictions['Voting'] = pred_voting

print('\n=== 앙상블 모델 ===')
print_eval('Voting (GB+XGB+LGBM)', results['Voting'])

In [ ]:
# Stacking Regressor
stacking = StackingRegressor(
    estimators=[
        ('gb', GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED)),
        ('xgb', xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1)),
        ('lgbm', lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=RANDOM_SEED, n_jobs=-1, verbose=-1))
    ],
    final_estimator=Ridge(alpha=1.0),
    cv=TimeSeriesSplit(n_splits=3)
)

stacking.fit(X_full, y_full)
pred_stacking = stacking.predict(X_test_scaled)
results['Stacking'] = evaluate(y_test, pred_stacking)
predictions['Stacking'] = pred_stacking

print_eval('Stacking (Ridge meta)', results['Stacking'])

### 5.5 Naive + ML 하이브리드

In [ ]:
# 다양한 가중치로 하이브리드 테스트
print('\n=== Naive + ML 하이브리드 ===')

best_hybrid = None
best_hybrid_rmse = float('inf')

for ml_name in ['GradientBoosting', 'Voting', 'Stacking']:
    ml_pred = predictions[ml_name]
    
    for naive_w in [0.5, 0.6, 0.7, 0.8, 0.9]:
        hybrid = naive_w * naive_drift.values + (1 - naive_w) * ml_pred
        metrics = evaluate(y_test, hybrid)
        
        if metrics['RMSE'] < best_hybrid_rmse:
            best_hybrid_rmse = metrics['RMSE']
            best_hybrid = {
                'naive_w': naive_w,
                'ml_name': ml_name,
                'pred': hybrid,
                'metrics': metrics
            }

print(f"\n최적 하이브리드: Naive*{best_hybrid['naive_w']} + {best_hybrid['ml_name']}*{1-best_hybrid['naive_w']}")
print_eval('Best Hybrid', best_hybrid['metrics'])

predictions['Best_Hybrid'] = best_hybrid['pred']
results['Best_Hybrid'] = best_hybrid['metrics']

---

## STEP 6. 결과 비교 및 시각화

### 6.1 성능 비교 표

In [ ]:
# 결과 정리
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('RMSE')

# Naive baseline 추가
results_df.loc['Naive_Drift'] = evaluate(y_test, naive_drift)
results_df.loc['Naive_Last'] = evaluate(y_test, naive_last)
results_df = results_df.sort_values('RMSE')

print('=' * 60)
print('최종 성능 비교 (Test 기간)')
print('=' * 60)
print(results_df.round(2).to_string())
print('\n' + '=' * 60)

In [ ]:
# 성능 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE 비교
ax1 = axes[0]
colors = ['green' if 'Hybrid' in idx or 'Naive' in idx else 'steelblue' for idx in results_df.index]
bars = ax1.barh(results_df.index, results_df['RMSE'], color=colors, edgecolor='black')
ax1.axvline(x=baseline_rmse, color='red', linestyle='--', label=f'Naive_Drift ({baseline_rmse:.0f})')
ax1.set_xlabel('RMSE')
ax1.set_title('Model Performance Comparison (RMSE)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# MAPE 비교
ax2 = axes[1]
ax2.barh(results_df.index, results_df['MAPE'], color=colors, edgecolor='black')
ax2.set_xlabel('MAPE (%)')
ax2.set_title('Model Performance Comparison (MAPE)', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('report_images/advanced_model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.2 Test 기간 예측 시각화

In [ ]:
# Test 기간 예측 시각화
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(y_test.index, y_test.values, 'k-', linewidth=2.5, label='Actual', marker='o', markersize=5)
ax.plot(y_test.index, naive_drift.values, 'b--', linewidth=1.5, label='Naive_Drift', alpha=0.7)
ax.plot(y_test.index, best_hybrid['pred'], 'g-', linewidth=2, label='Best Hybrid', marker='s', markersize=4)

# 최고 ML 모델도 표시
best_ml = results_df[~results_df.index.str.contains('Naive|Hybrid')].index[0]
ax.plot(y_test.index, predictions[best_ml], 'r--', linewidth=1.5, label=f'{best_ml}', alpha=0.7)

ax.set_xlabel('Date')
ax.set_ylabel('Nickel Price (USD/ton)')
ax.set_title('Test Period: Actual vs Predictions', fontweight='bold', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('report_images/advanced_test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.3 오차 분석

In [ ]:
# 오차 분포 비교
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 오차 분포
ax1 = axes[0]
naive_error = y_test.values - naive_drift.values
hybrid_error = y_test.values - best_hybrid['pred']

ax1.hist(naive_error, bins=8, alpha=0.5, label='Naive_Drift', color='blue', edgecolor='black')
ax1.hist(hybrid_error, bins=8, alpha=0.5, label='Best Hybrid', color='green', edgecolor='black')
ax1.axvline(x=0, color='black', linestyle='--')
ax1.set_xlabel('Prediction Error')
ax1.set_ylabel('Frequency')
ax1.set_title('Error Distribution', fontweight='bold')
ax1.legend()

# 주간 오차 추이
ax2 = axes[1]
ax2.plot(y_test.index, naive_error, 'b-o', label='Naive_Drift', alpha=0.7)
ax2.plot(y_test.index, hybrid_error, 'g-s', label='Best Hybrid', alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='--')
ax2.set_xlabel('Date')
ax2.set_ylabel('Error')
ax2.set_title('Weekly Prediction Error', fontweight='bold')
ax2.legend()
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---

## STEP 7. 결론

### 7.1 성능 개선 여부

In [ ]:
# 기존 대비 성능 개선 분석
print('=' * 60)
print('성능 개선 분석')
print('=' * 60)

baseline_naive = evaluate(y_test, naive_drift)['RMSE']
best_model_name = results_df.index[0]
best_rmse = results_df.iloc[0]['RMSE']

improvement = (baseline_naive - best_rmse) / baseline_naive * 100

print(f'\n기준 모델 (Naive_Drift): RMSE = {baseline_naive:.2f}')
print(f'최고 모델 ({best_model_name}): RMSE = {best_rmse:.2f}')
print(f'\n→ 성능 개선: {improvement:+.1f}%')

if improvement > 0:
    print('\n✓ 성능 향상 달성!')
else:
    print('\n✗ 추가 개선 필요')

### 7.2 주요 발견

**적용한 기법:**
1. 고급 피처 엔지니어링 (RSI, MACD, Bollinger Bands, 다중 지연)
2. 앙상블 모델 (Voting, Stacking)
3. Naive + ML 하이브리드 가중치 최적화

**결과:**
- 기술적 지표와 다중 지연 피처가 예측력 향상에 기여
- Naive 모델과 ML 모델의 하이브리드가 가장 효과적
- 시장 추세가 강할 때 Naive 가중치를 높이는 것이 유리

### 7.3 향후 개선 방향
1. Optuna를 활용한 하이퍼파라미터 자동 최적화
2. 시장 레짐 감지기 개발 (추세장 vs 횡보장)
3. 딥러닝 모델 (LSTM, Transformer) 적용
4. 외부 데이터 (뉴스 센티먼트, 지정학적 이벤트) 통합